In [ ]:
export default {
  name: "table",
  data() {
  // 使用 data 对象可以很方便地管理组件中所需的数据，并且能够保证组件的响应性能力
    return {
      showIframe: false,
      iframeUrl: '',
      diaLogShow: false,
      filename1: '',
      filename2: '',
      filename3: '',

      sheet1: [],
      sheet2: [],
      sheet3: [],
      selectedTheme: 'westeros',
      theme: [
        "light",
        "dark",
        "white",
        "chalk",
        "essos",
        "infographic",
        "macarons",
        "purple-passion",
        "roma",
        "romantic",
        "shine",
        "vintage",
        "walden",
        "westeros",
        "wonderland",
        "halloween",
      ],
      selectedSheet1: '',


      showTableData1: [],
      showTableHeader1: [ // 更改显示在前端的表格，如果是一个表格则只有以下设置。多个表格类似设置。
          { code: "date", name: "姓名" },
          { code: "date", name: "学号" },
          { code: "date", name:"SPOC成绩" },
          { code: "date", name: "雨课堂成绩（课前测，课中、课后测）" },
          { code: "date",  name: "PTA作业" },
          { code: "date", name: "PTA测试" },
          { code: "date", name: "project成绩" },
          { code: "date", name: "平时总成绩" },
          { code: "date", name: "终结性成" },
          { code: "date", name: "总评成绩" }
      ],
      showTableData2: [],
      showTableHeader2: ['暂无数据'],
      showTableData3: [],
      showTableHeader3: ['暂无数据'],
      exportFileName: '',
    }
  },
  methods: {
    handleProcess(){
      // 处理excel，用get提交一个请求。三个数组来分别存储三个表格的表头数据
      const baseUrl = process.env.VUE_APP_BASE_URL // 设置为url=vies9?filename1=this.filename1
      const url = baseUrl + '/merge?filename1=' + this.filename1 + '&filename2=' + this.filename2
      axios.get(url).then(response => {//返回的数据只有一个表格数据。
        this.showTableHeader1 =  response.data.data.headers
        this.showTableData1 =  response.data.data.data
        this.showTableHeader2 = []
        this.showTableData2 = []
        this.filename =  ''
        this.exportFileName =  response.data.data.filename
      })

    },
    handlePreviewCharts() {
      //点击绑定的查看图片，这段代码的功能是在新窗口中打开一个URL
      const baseUrl = process.env.VUE_APP_BASE_URL // 设置处理图片的函数
      const url = baseUrl + '/export_dimension_radar_chart?filename1='
          + this.filename1 + '&sheet1=' + this.selectedSheet1 + '&theme=' + this.selectedTheme
      window.open(url) //在新窗口打开结果，以url的形式传递
      return
      console.log('url :', url)
      this.showIframe = true
      this.iframeUrl = url

    },
    handlePreview() {
      //点击绑定的handlePreview，导入Excel可以查看数据
      if (this.sheet1 == '' ) {
        alert('请先选择sheet')
        return
      }
      const baseUrl = process.env.VUE_APP_BASE_URL
      const url = baseUrl + '/get_one_excel_data?filename1='
          + this.filename1 + '&sheet1=' + this.selectedSheet1
      console.log('url:', url)

      this.showTableData1 =  []
      this.showTableHeader1=[]
      axios.get(url,).then(response => {
        // 三个数组来分别存储三个表格的表头数据，使用方法：this.showTableHeader1 = response.data.data.headers;
        console.log(response.data.data.data1.data)
        this.showTableData1 =  response.data.data.data1.data.slice(1)
        this.showTableHeader1 =  response.data.data.data1.headers.slice(1)
          }
      )
    },
    submit() {
      //点击提交之后的函数处理
      const baseUrl = process.env.VUE_APP_BASE_URL
      const url = baseUrl + '/upload_file_and_get_sheet_name'
      let formData = new FormData();
      const files1 = this.$refs.upload1.uploadFiles
      if (files1.length === 0 ) {
        alert('文件为空')
        return
      }
      let file1 = files1[0].raw
      formData.append('file1', file1)
      console.log(url);
      // 添加请求头
      axios.post(url,formData,{
        'Content-type' : 'multipart/form-data'
      })
          .then(response => {
            console.log(response.data)
            if (response.data.code === 200) {
              // this.showTableHeader1 =  response.data.data.data1.headers
              // this.showTableData1 =  response.data.data.data1.data
              this.filename1 =  response.data.data.data1.filename
              this.sheet1 = response.data.data.data1.sheet

              this.exportFileName = ''
              this.$refs.upload1.uploadFiles = []
              this.diaLogShow = false

            } else {
              alert(response.data.msg)
            }
          })

    },

    handleAdd() {
      //点击选择Excel
      this.form = {}
      this.diaLogShow = true
      this.$refs.upload1.uploadFiles = []
    },
  },
  mounted() {
  }
}

In [ ]:
总共包含4个方法，分别是
handleAdd
submit
handpreview
handpreviewcharts
提交获取sheet的名字就不用了，所有都是通用的。下面是涉及到的后端
对应的前端事件。分别都是前端涉及到与后端的能够返回事件处理的结果。一个页面就有4个接口，4次输入，读取接口输入，读取sheet参数，读取图片参数。
<el-button type="primary" @click="submit">确 定</el-button>
<el-button type="primary" icon="el-icon-search" @click="handlePreviewCharts" style="margin: 0px 0px 0px 10px">再预览图片</el-button>
<el-button type="success" icon="el-icon-upload" @click="handleAdd" style="margin: 0px 0px 0px 10px">选择Excel</el-button>
<el-button type="primary" icon="el-icon-search" @click="handlePreview" style="margin: 0px 0px 0px 10px">先预览数据</el-button>

In [ ]:
图形代码部分
自定义函数：
from dimension_radar_chart_function_8 import dimension_radar_chart
# 维度图调用得到文件名字符串，然后导出html
@request_mapping('/export_dimension_radar_chart')
async def export_dimension_radar_chart(self):
    self.set_header('Access-Control-Allow-Origin', '*')  # 跨域
    filename1 = self.get_argument('filename1')
    sheet1 = self.get_argument('sheet1')
    theme = self.get_argument('theme')
    output_html = dimension_radar_chart(filename1, sheet1, theme)
    # print(output_html) 生成的文件
    self.write(output_html)


获取dimension_radar_chart函数的使用，from文件名dimension_radar_chart_function_8.py导入dimension_radar_chart
    
from pyecharts import options as opts
from pyecharts.charts import Radar
from pyecharts.globals import CurrentConfig
CurrentConfig.ONLINE_HOST = "https://cdn.jsdelivr.net/npm/echarts@5.2.2/dist/"
from pyecharts.globals import ThemeType
import pandas as pd

def dimension_radar_chart(file, sheet1, theme):
    返回图片

In [ ]:
const url = baseUrl + '/get_one_excel_data?filename1='
          + this.filename1 + '&sheet1=' + this.selectedSheet1

    
调用excel获取函数的方法。返回获取的data1，header等等。此函数在run.py下面。
@request_mapping('/get_one_excel_data', method='get')
async def get_one_excel_data(self):
    """预览"""
    self.set_header('Access-Control-Allow-Origin', '*')  # 跨域
    filename1 = self.get_argument('filename1')
    sheet1 = self.get_argument('sheet1')
    header1, data1 = _load_excel_data(filename1, sheet1)
    self.write({
        'code': 200,
        'data': {
            'data1': {
                'headers': header1,
                'data': data1,
                'filename': filename1,
            },
        }
    })

读取excell数据函数
def _load_excel_data(filename, sheet_name=None):

In [ ]:
const url = baseUrl + '/upload_file_and_get_sheet_name'
上传获取一个sheet的名字和数据。

In [ ]:
获取数据假设都已经好了，现在假设完成表图的函数。当到处不知道该怎么解决问题的时候，尝试低下头，安静下来，看看该怎么做？
按流程走，原来的流程停止在什么地方，然后再看api哪个地方不对。